In [1]:
from utils import *
import pandas as pd

In [2]:
df_mme = pd.read_csv("data/M.csv")

In [3]:
df_mme.query("Estado != 'R'")

,Identificador,Fecha,Estado,Tipo,Cuenta,Beneficiario,Categoría,Importe,Divisa,Número,Notas


In [4]:
df = df_mme.pipe(clean_data).query("Fecha >= '2022-01-01'")
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [5]:
df.sample(5)

,Fecha,Tipo,Categoria,Subcategoria,Beneficiario,Transaccion,Importe
1596,2022-03-16,Gastos,Despacho - Ocio,Restaurantes,VIENA CAPELLANES,Cargo,-3.2
385,2022-09-08,Gastos,Gastos Fijos,Paga,CLAUDIA,Cargo,-60.0
283,2022-06-20,Gastos,Supervivencia,Farmacia,No Definido,Cargo,-5.88
1674,2022-04-25,Gastos,Supervivencia,Alimentación,FRUTERIA ALCORCON,Cargo,-14.89
1499,2022-01-05,Gastos,Extras,Regalos,CARREFOUR,Cargo,-30.0


In [6]:
(df_hogar
    .pipe(pivot_by_category_totals, ["Tipo"])
       .pipe(style_locale_es)
 )

,Tipo,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,"-9.524,11","-8.505,44","-12.485,53","-5.234,44","-3.461,10","-8.288,79","-7.771,75","-5.210,00","-5.021,79","-5.208,63","-4.370,48","-4.858,84","-6.661,74","-79.940,90"
1,Ingresos,"10.853,84","10.296,45","9.266,42","2.727,39","2.587,06","5.291,84","9.110,07","5.627,59","5.602,14","7.735,52","9.696,62","2.642,34","6.786,44","81.437,28"
2,Total,"1.329,73","1.791,01","-3.219,11","-2.507,05","-874,04","-2.996,95","1.338,32","417,59","580,35","2.526,89","5.326,14","-2.216,50","124,70","1.496,38"


In [7]:
(df_despacho
     .pipe(pivot_by_category_totals, ["Tipo"])
     .pipe(style_locale_es)
 )

,Tipo,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,"-492,56","-490,46","-2.310,61","-440,76","-283,38","-1.430,33","-431,15","-593,03","-1.782,86","-638,43","-655,18","-627,95","-848,06","-10.176,70"
1,Ingresos,"1.938,61","4.446,89","8.752,27","242,00","50,00","3.903,71","3.670,00","3.413,30","4.578,71","5.334,69","3.290,60","410,00","3.335,90","40.030,78"
2,Total,"1.446,05","3.956,43","6.441,66","-198,76","-233,38","2.473,38","3.238,85","2.820,27","2.795,85","4.696,26","2.635,42","-217,95","2.487,84","29.854,08"
